In [ ]:
# Q3 a) my homework 
class SudokuCSP:
    def __init__(self, puzzle):
        self.puzzle = puzzle  
        self.variables = range(81)
        self.domains = self.initialize_domains()
        self.constraints = self.define_constraints()

    def initialize_domains(self):
        domains = {}
        for i in self.variables:
            if self.puzzle[i] == '0':
                domains[i] = set(range(1, 10))  
            else:
                domains[i] = {int(self.puzzle[i])} 
        return domains

    def define_constraints(self):
        constraints = []
       
        for i in range(9):
            row_vars = [i * 9 + j for j in range(9)]
            constraints.extend(self.all_different_constraint(row_vars))

        
        for j in range(9):
            col_vars = [i * 9 + j for i in range(9)]
            constraints.extend(self.all_different_constraint(col_vars))

       
        for row_start in range(0, 9, 3):
            for col_start in range(0, 9, 3):
                square_vars = [
                    (row_start + i) * 9 + (col_start + j)
                    for i in range(3)
                    for j in range(3)
                ]
                constraints.extend(self.all_different_constraint(square_vars))
        return constraints

    def all_different_constraint(self, variables):
        
        return [(v1, v2) for i, v1 in enumerate(variables) for v2 in variables[i+1:]]

    def is_consistent(self, assignment, variable, value):
       
        for neighbor in self.get_neighbors(variable):
            if neighbor in assignment and assignment[neighbor] == value:
                return False
        return True

    def get_neighbors(self, variable):
       
        neighbors = set()

        
        row = variable // 9
        neighbors.update([row * 9 + j for j in range(9) if row * 9 + j != variable])

        
        col = variable % 9
        neighbors.update([i * 9 + col for i in range(9) if i * 9 + col != variable])

        
        row_start = (variable // 9 // 3) * 3
        col_start = (variable % 9 // 3) * 3
        neighbors.update([
            (row_start + i) * 9 + (col_start + j)
            for i in range(3)
            for j in range(3)
            if (row_start + i) * 9 + (col_start + j) != variable
        ])

        return neighbors

    def ac3(self):
        queue = list(self.constraints)  

        while queue:
            (x, y) = queue.pop(0)
            if self.revise(x, y):
                if not self.domains[x]:
                    return False  
                for neighbor in self.get_neighbors(x):
                    if (neighbor, x) in self.constraints:
                        queue.append((neighbor, x))
        return True

    def revise(self, x, y):
        revised = False
        to_remove = set()
        for val_x in self.domains[x]:
            satisfies_constraint = False
            for val_y in self.domains[y]:
                if val_x != val_y:
                    satisfies_constraint = True
                    break
            if not satisfies_constraint:
                to_remove.add(val_x)
                revised = True
        self.domains[x] -= to_remove
        return revised

    def backtracking_search(self):
        return self.recursive_backtracking({})

    def recursive_backtracking(self, assignment):
        if len(assignment) == 81:
            return assignment  

        variable = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(variable):
            if self.is_consistent(assignment, variable, value):
                assignment[variable] = value
               
                old_domains = {k: self.domains[k].copy() for k in self.domains}
                self.domains[variable] = {value} 

                if self.ac3():
                    result = self.recursive_backtracking(assignment)
                    if result:
                        return result

                
                self.domains = old_domains
                del assignment[variable] 

        return False  # Failure

    def select_unassigned_variable(self, assignment):
        
        unassigned = [v for v in self.variables if v not in assignment]
        return min(unassigned, key=lambda var: len(self.domains[var]))

    def order_domain_values(self, variable):
       
        return sorted(self.domains[variable], key=lambda value: self.count_conflicts(variable, value))

    def count_conflicts(self, variable, value):
      
        conflicts = 0
        for neighbor in self.get_neighbors(variable):
            if neighbor in self.domains and value in self.domains[neighbor]:
                conflicts += 1
        return conflicts

    def solve(self):

        if not self.ac3():
            return None  

        solution = self.backtracking_search()
        if solution:
            return "".join(str(solution[i]) for i in range(81))
        else:
            return None

def solve_sudoku_csp(puzzle):
    csp = SudokuCSP(puzzle)
    solution = csp.solve()
    return solution

puzzle_string = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"
solution = solve_sudoku_csp(puzzle_string)

if solution:
    print(solution)
else:
    print("No solution found.")


483921657967345821251876493548132976729564138136798245372689514814253769695417382


In [ ]:
# Q3 b) Google OR-Tools Implementation: 
from ortools.sat.python import cp_model
import time

def solve_sudoku_ortools(grid_string):
    """Solves a Sudoku puzzle using Google OR-Tools."""

    model = cp_model.CpModel()

    # Create variables
    grid = {}
    for i in range(9):
        for j in range(9):
            grid[(i, j)] = model.NewIntVar(1, 9, f'grid_{i}_{j}')

    # Add constraints
    # Row constraints
    for i in range(9):
        model.AddAllDifferent([grid[(i, j)] for j in range(9)])

    # Column constraints
    for j in range(9):
        model.AddAllDifferent([grid[(i, j)] for i in range(9)])

    # 3x3 block constraints
    for block_row in range(3):
        for block_col in range(3):
            cells = [
                grid[(block_row * 3 + i, block_col * 3 + j)]
                for i in range(3)
                for j in range(3)
            ]
            model.AddAllDifferent(cells)

    # Initial values
    for i in range(9):
        for j in range(9):
            if grid_string[i * 9 + j] != '0':
                model.Add(grid[(i, j)] == int(grid_string[i * 9 + j]))

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Extract the solution
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        solution = ""
        for i in range(9):
            for j in range(9):
                solution += str(solver.Value(grid[(i, j)]))
        return solution
    else:
        return None  # No solution found

# Example Usage
puzzle_string = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"
start_time = time.time()
solution = solve_sudoku_ortools(puzzle_string)
end_time = time.time()

if solution:
    print(f"OR-Tools Solution: {solution}")
    print(f"OR-Tools Time: {end_time - start_time:.4f} seconds")
else:
    print("OR-Tools: No solution found.")


load C:\Users\cycle\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\cycle\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\cycle\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\cycle\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ortools\.libs\re2.dll...
load C:\Users\cycle\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\cycle\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python31

In [6]:
# ChatGPT Sudoku Solver 
def solve_sudoku_chatgpt(puzzle_str):
    """
    Solve a Sudoku puzzle using backtracking and AC-3 constraint propagation.

    :param puzzle_str: String of 81 digits (0 represents an empty cell)
    :return: String of 81 digits representing the solved puzzle
    """
    assert len(puzzle_str) == 81

    # Constants
    digits = '123456789'
    rows = 'ABCDEFGHI'
    cols = digits
    squares = [r + c for r in rows for c in cols]

    # Units and peers
    def cross(a, b):
        return [s + t for s in a for t in b]

    unitlist = (
        [cross(r, cols) for r in rows] +
        [cross(rows, c) for c in cols] +
        [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
    )
    units = {s: [u for u in unitlist if s in u] for s in squares}
    peers = {s: set(sum(units[s], [])) - {s} for s in squares}

    # Parse input string into a dict of possible values
    def parse_grid(grid):
        values = {s: digits for s in squares}
        for s, d in zip(squares, grid):
            if d in digits and not assign(values, s, d):
                return False
        return values

    # Assign a value to a square and propagate constraints
    def assign(values, s, d):
        other_values = values[s].replace(d, '')
        if all(eliminate(values, s, d2) for d2 in other_values):
            return values
        else:
            return False

    # Eliminate a value from a square and enforce arc consistency
    def eliminate(values, s, d):
        if d not in values[s]:
            return values  # Already eliminated

        values[s] = values[s].replace(d, '')

        # If a square has no values, fail
        if len(values[s]) == 0:
            return False

        # If only one value remains, eliminate it from peers
        elif len(values[s]) == 1:
            d2 = values[s]
            if not all(eliminate(values, s2, d2) for s2 in peers[s]):
                return False

        # If a unit has only one place for a digit, assign it
        for u in units[s]:
            dplaces = [s for s in u if d in values[s]]
            if len(dplaces) == 0:
                return False
            elif len(dplaces) == 1:
                if not assign(values, dplaces[0], d):
                    return False

        return values

    # AC-3 Algorithm
    def ac3(values):
        queue = [(xi, xj) for xi in squares for xj in peers[xi]]
        while queue:
            xi, xj = queue.pop()
            if revise(values, xi, xj):
                if len(values[xi]) == 0:
                    return False
                for xk in peers[xi] - {xj}:
                    queue.append((xk, xi))
        return values

    def revise(values, xi, xj):
        revised = False
        for d in values[xi]:
            if len(values[xj]) == 1 and values[xj] == d:
                values[xi] = values[xi].replace(d, '')
                revised = True
        return revised

    # Recursive backtracking search
    def backtrack(values):
        if values is False:
            return False
        if all(len(values[s]) == 1 for s in squares):
            return values

        # Choose the unfilled square with the fewest possibilities
        n, s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
        for d in values[s]:
            new_values = values.copy()
            if assign(new_values, s, d):
                result = backtrack(ac3(new_values))
                if result:
                    return result
        return False

    # Start solving
    values = parse_grid(puzzle_str)
    if not values:
        return None

    result = backtrack(ac3(values))
    if result:
        return ''.join(result[s] for s in squares)
    else:
        return None


In [ ]:
import time

def time_sudoku_solver(solver_function, puzzle, solver_name):
    start_time = time.time()
    solution = solver_function(puzzle)
    end_time = time.time()
    duration = end_time - start_time

    print(f"{solver_name}:")
    if solution:
        print(f"  Solution: {solution}")
        print(f"  Time: {duration:.4f} seconds")
    else:
        print("  No solution found.")
    return duration


puzzle_string = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"

my_solver_time = time_sudoku_solver(solve_sudoku_csp, puzzle_string, "My CSP Solver")
ortools_time = time_sudoku_solver(solve_sudoku_ortools, puzzle_string, "OR-Tools Solver")
chatgpt_time = time_sudoku_solver(solve_sudoku_chatgpt, puzzle_string, "ChatGPT Solver")

print("\nTime Comparisons:")
print(f"  My CSP Solver: {my_solver_time:.4f} seconds")
print(f"  OR-Tools Solver: {ortools_time:.4f} seconds")
print(f"  ChatGPT Solver: {chatgpt_time:.4f} seconds")

# Improving My CSP Solver (Based on ChatGPT Insights):

# More Efficient AC-3: I'll analyze the ChatGPT solver's eliminate() function to see if I can adapt any of its techniques to make my revise() function more efficient. Specifically, I'll look at how it handles the case where a unit has only one place for a digit.
# Dynamic MRV: I'll implement a dynamic MRV heuristic, where the MRV is recalculated after each assignment. This can help to choose the most constrained variable at each step.

My CSP Solver:
  Solution: 483921657967345821251876493548132976729564138136798245372689514814253769695417382
  Time: 1.9049 seconds
OR-Tools Solver:
  Solution: 483921657967345821251876493548132976729564138136798245372689514814253769695417382
  Time: 0.0318 seconds
ChatGPT Solver:
  Solution: 483921657967345821251876493548132976729564138136798245372689514814253769695417382
  Time: 0.0000 seconds

Time Comparisons:
  My CSP Solver: 1.9049 seconds
  OR-Tools Solver: 0.0318 seconds
  ChatGPT Solver: 0.0000 seconds


In [8]:
# q1 

def query(x):
    return -1 * (x - 7)**2 + 49

def find_peak(N: int) -> int:
 

    current_position = N // 2  
    step_size = N // 4 

    while step_size > 0:
        left = max(0, current_position - step_size)
        right = min(N, current_position + step_size)

        current_elevation = query(current_position)
        left_elevation = query(left)
        right_elevation = query(right)

        if left_elevation > current_elevation:
            current_position = left
        elif right_elevation > current_elevation:
            current_position = right
        else:
            step_size //= 2 

    return current_position


In [ ]:
# q2 
import random


NUM_TASKS = 7
NUM_FACILITIES = 3
POPULATION_SIZE = 6
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.2
NUM_GENERATIONS = 100

TASK_TIMES = [5, 8, 4, 7, 6, 3, 9]
FACILITY_CAPACITIES = [24, 30, 28]
COST_MATRIX = [
    [10, 12, 9],
    [15, 14, 16],
    [8, 9, 7],
    [12, 10, 13],
    [14, 13, 12],
    [9, 8, 10],
    [11, 12, 13]
]

def create_chromosome():
    
    return [random.randint(1, NUM_FACILITIES) for _ in range(NUM_TASKS)]

def calculate_fitness(chromosome):
    
    total_cost = 0
    facility_times = [0] * NUM_FACILITIES
    penalty = 0

    for i in range(NUM_TASKS):
        facility = chromosome[i] - 1  
        task_time = TASK_TIMES[i]
        cost = COST_MATRIX[i][facility]

        total_cost += task_time * cost
        facility_times[facility] += task_time

    
    for i in range(NUM_FACILITIES):
        if facility_times[i] > FACILITY_CAPACITIES[i]:
            penalty += (facility_times[i] - FACILITY_CAPACITIES[i]) * 10000 

    if total_cost + penalty == 0:
        return 0  
    return 1 / (total_cost + penalty)

def roulette_wheel_selection(population, fitness_values):
   
    total_fitness = sum(fitness_values)
    if total_fitness == 0:
       
        
        return random.choice(population)  
    
    probabilities = [f / total_fitness for f in fitness_values]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    
    rand = random.random()
    for i, cum_prob in enumerate(cumulative_probabilities):
        if rand <= cum_prob:
            return population[i]
    return population[-1]  

def one_point_crossover(parent1, parent2):
  
    if random.random() < CROSSOVER_RATE:
        crossover_point = random.randint(1, NUM_TASKS - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    else:
        return parent1[:], parent2[:]  

def swap_mutation(chromosome):
    if random.random() < MUTATION_RATE:
        pos1, pos2 = random.sample(range(NUM_TASKS), 2)
        chromosome[pos1], chromosome[pos2] = chromosome[pos2], chromosome[pos1]
    return chromosome[:] 


def genetic_algorithm():
   
    population = [create_chromosome() for _ in range(POPULATION_SIZE)]

    for generation in range(NUM_GENERATIONS):
        fitness_values = [calculate_fitness(chromosome) for chromosome in population]

        
        new_population = []
        for _ in range(POPULATION_SIZE // 2): 
            parent1 = roulette_wheel_selection(population, fitness_values)
            parent2 = roulette_wheel_selection(population, fitness_values)

            
            child1, child2 = one_point_crossover(parent1, parent2)

            
            child1 = swap_mutation(child1)
            child2 = swap_mutation(child2)

            new_population.extend([child1, child2])

        population = new_population

    
    fitness_values = [calculate_fitness(chromosome) for chromosome in population]
    best_index = fitness_values.index(max(fitness_values))
    best_chromosome = population[best_index]
    best_fitness = fitness_values[best_index]

    return best_chromosome, best_fitness


best_solution, best_fitness = genetic_algorithm()

print("Best Task Allocation:", best_solution)
print("Best Fitness:", best_fitness)


total_cost = 0
facility_times = [0] * NUM_FACILITIES
for i in range(NUM_TASKS):
    facility = best_solution[i] - 1
    task_time = TASK_TIMES[i]
    cost = COST_MATRIX[i][facility]
    total_cost += task_time * cost
    facility_times[facility] += task_time

print("Total Cost:", total_cost)
print("Facility Times:", facility_times)


Best Task Allocation: [3, 3, 2, 2, 3, 1, 1]
Best Fitness: 0.0020964360587002098
Total Cost: 477
Facility Times: [12, 11, 19]
